Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [ ]:
import retinoto_py as fovea
model_name = 'convnext_base'
args = fovea.Params(do_fovea=True, model_name=model_name, subset_factor=1)
args

Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-7.5, padding_mode='zeros', seed=2025, batch_size=80, num_workers=1, in_memory=False, model_name='convnext_base', num_epochs=41, subset_factor=1, lr=2e-07, delta1=0.2, delta2=0.007, weight_decay=0.003, label_smoothing=0.05, shuffle=True, verbose=False)

# testing different networks

In [2]:
dataset = 'bbox'
model_filename = args.data_cache /  f'32_fovea_model_name={model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)

In [3]:
args = fovea.Params()

json_filename = args.data_cache / '33_model_comparison.json'

%rm {json_filename}  # FORCING RECOMPUTE

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results = []
    VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
    val_dataset = fovea.get_dataset(args, VAL_DATA_DIR)
    val_loader = fovea.get_loader(args, val_dataset)

    model = fovea.load_model(args)
    param_stats = fovea.count_parameters(model)
    total_layers = fovea.count_layers(model)
    tic = fovea.time.time()
    accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Model {model_name}\t dataset: {dataset}")            
    toc = fovea.time.time()
    
    this_result = { 'model_name': model_name,
                    'dataset': dataset,
                    'accuracy': accuracy,
                    'wall_clock_time': (toc-tic)/len(val_dataset),
                    'total_parameters': param_stats['total_parameters'],
                    'trainable_parameters': param_stats['trainable_parameters'],
                    'total_layers': total_layers}
    all_results.append(this_result)
    results = fovea.pd.DataFrame(all_results)
    results.to_json(json_filename, orient='records', indent=2)


Model convnext_base	 dataset: bbox:   0%|          | 0/1005 [00:00<?, ?it/s]

In [4]:
results

,model_name,dataset,accuracy,wall_clock_time,total_parameters,trainable_parameters,total_layers
0,convnext_base,bbox,0.782799,0.006036,88591464,88591464,383


In [5]:
# fig, ax = fovea.plot_model_comparison(results, fovea.params.all_model_names, fovea.params.all_datasets, do_masks=[True, False], figures_folder=args.figures_folder, save_name='33_model_comparison', exts=['svg'])
